## Training

In [ ]:
import os
os.chdir("/Users/pavankumar/Projects/CNN-Classifier-End-to-end/")
%pwd

## Entity

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

@dataclass(frozen = True)
class CallbackConfig:
    root_dir: Path
    tensorflow_root_log_dir: Path
    checkpoint_model_filepath: Path


## Configuration Manager

In [ ]:
from CNN_CLASSIFIER.constants import *
from CNN_CLASSIFIER.utils import read_yaml, create_directories
import tensorflow as tf


In [ ]:

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_prepare_callback_config(self) -> CallbackConfig:
        config = self.config.callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorflow_root_log_dir)
        ])

        prepare_callback_config = CallbackConfig(
            root_dir=Path(config.root_dir),
            tensorflow_root_log_dir=Path(config.tensorflow_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "PetImages")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )
        return training_config



## Components

#### Callbacks Components

In [ ]:
import tensorflow as tf
import time

class PrepareCallback:
    def __init__(self, config: CallbackConfig):
        self.config = config

    @property
    def _create_tb_callback(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
           self.config.tensorflow_root_log_dir,
           f"tb_logs_at :: {timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir = tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),
            save_best_only=True
        )
    
    def get_tb_ckpt_callbacks(self):
      return [
         self._create_tb_callback,
         self._create_ckpt_callbacks
         ]
    

    


#### Training Components

In [ ]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [51]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
      
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    def training_valid_generation(self):
        
        datagenerated_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.20,
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation="bilinear",
           
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
           **datagenerated_kwargs
        )
        
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )
         
        if self.config.params_is_augmentation:
           train_datagenerated = tf.keras.preprocessing.image.ImageDataGenerator(
              rotation_range = 40,
              horizontal_flip = True,
              width_shift_range = 0.2,
              height_shift_range = 0.2,
              shear_range = 0.2,
              zoom_range = 0.2,
              ** datagenerated_kwargs
           )
        else:
            train_datagenerated = valid_datagenerator
        
        self.train_generator = train_datagenerated.flow_from_directory(
            directory= self.config.training_data,
            subset = "training",
            shuffle = True,
            **dataflow_kwargs
        )
    
    @staticmethod
    def save_model(path : Path, model: tf.keras.Model):
        model.save(path)
        
    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples//self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples//self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs = self.config.params_epochs,
            steps_per_epoch = self.steps_per_epoch,
            validation_data = self.valid_generator,
            validation_steps = self.validation_steps,
            callbacks = callback_list
        )

        self.save_model(
            path = self.config.trained_model_path,
            model = self.model
        )

## Pipeline

In [52]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config =  config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config = prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config = training_config)
    training.get_base_model()
    training.training_valid_generation()
    training.train(
        callback_list = callback_list
    )

except Exception as e:
    raise(e)

[2024-05-02 18:13:13,296: INFO: common: yaml file:config/config.yaml loaded Successfully]
[2024-05-02 18:13:13,297: INFO: common: yaml file:params.yaml loaded Successfully]
[2024-05-02 18:13:13,298: INFO: common: Created Directory at: artifacts]
[2024-05-02 18:13:13,298: INFO: common: Created Directory at: artifacts/callbacks/checkpoint_dir]
[2024-05-02 18:13:13,299: INFO: common: Created Directory at: artifacts/callbacks/tensorflow_log_dir]
[2024-05-02 18:13:13,299: INFO: common: Created Directory at: artifacts/training]
Found 78 images belonging to 2 classes.
Found 318 images belonging to 2 classes.
19/19 [==============================] - 1008s 56s/step - loss: 18.6007 - accuracy: 0.4768 - val_loss: 1.2366 - val_accuracy: 0.5469


/Users/pavankumar/anaconda3/envs/cnncls/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
